<a href="https://colab.research.google.com/github/stalgiag/DAIN/blob/master/DAIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Check for CUDA

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


Install Pytorch

In [3]:
# different torch install needed
# !pip3 uninstall torch
!pip3 install -U https://download.pytorch.org/whl/cu100/torch-1.0.0-cp36-cp36m-linux_x86_64.whl

    100% |████████████████████████████████| 591.8MB 26kB/s 


Check Pytorch Install Works with Cuda

In [5]:
!python -c "import torch; print(torch.__version__); print(torch.cuda.is_available())"

1.0.0
True


In [6]:
# we will verify that GPU is enabled for this notebook
# following should print: CUDA is available!  Training on GPU ...
# 
# if it prints otherwise, then you need to enable GPU: 
# from Menu > Runtime > Change Runtime Type > Hardware Accelerator > GPU

import torch
import numpy as np

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [9]:
# check that gcc is installed
# should print version number along bottom
!gcc -v

Using built-in specs.
COLLECT_GCC=gcc
COLLECT_LTO_WRAPPER=/usr/lib/gcc/x86_64-linux-gnu/7/lto-wrapper
OFFLOAD_TARGET_NAMES=nvptx-none
OFFLOAD_TARGET_DEFAULT=1
Target: x86_64-linux-gnu
Configured with: ../src/configure -v --with-pkgversion='Ubuntu 7.3.0-27ubuntu1~18.04' --with-bugurl=file:///usr/share/doc/gcc-7/README.Bugs --enable-languages=c,ada,c++,go,brig,d,fortran,objc,obj-c++ --prefix=/usr --with-gcc-major-version-only --program-suffix=-7 --program-prefix=x86_64-linux-gnu- --enable-shared --enable-linker-build-id --libexecdir=/usr/lib --without-included-gettext --enable-threads=posix --libdir=/usr/lib --enable-nls --with-sysroot=/ --enable-clocale=gnu --enable-libstdcxx-debug --enable-libstdcxx-time=yes --with-default-libstdcxx-abi=new --enable-gnu-unique-object --disable-vtable-verify --enable-libmpx --enable-plugin --enable-default-pie --with-system-zlib --with-target-system-zlib --enable-objc-gc=auto --enable-multiarch --disable-werror --with-arch-32=i686 --with-abi=m64 --with-

Clone my modified fork

In [22]:
#clone repo - my fork has a few simple mods that make this work with colab
!git clone https://github.com/stalgiag/DAIN.git

Cloning into 'DAIN'...
remote: Enumerating objects: 160, done.
remote: Counting objects: 100% (160/160), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 463 (delta 102), reused 108 (delta 72), pack-reused 303
Receiving objects: 100% (463/463), 145.47 KiB | 6.61 MiB/s, done.
Resolving deltas: 100% (253/253), done.


Setup

In [23]:
#move into repo and setup
#may take awhile
%cd DAIN
%cd my_package
!./build.sh
%cd ../PWCNet/correlation_package_pytorch1_0
!./build.sh

/content/DAIN
/content/DAIN/my_package
Need pytorch>=1.0.0
./build.sh: line 4: activate: No such file or directory
running install
running bdist_egg
running egg_info
creating mindepthflowprojection_cuda.egg-info
writing mindepthflowprojection_cuda.egg-info/PKG-INFO
writing dependency_links to mindepthflowprojection_cuda.egg-info/dependency_links.txt
writing top-level names to mindepthflowprojection_cuda.egg-info/top_level.txt
writing manifest file 'mindepthflowprojection_cuda.egg-info/SOURCES.txt'
writing manifest file 'mindepthflowprojection_cuda.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_ext
building 'mindepthflowprojection_cuda' extension
creating build
creating build/temp.linux-x86_64-3.6
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.6/dist-packages/torch/lib/include -I/usr/loca

In [24]:
%cd ../../

/content/DAIN


In [0]:
!mkdir model_weights
!mkdir MiddleBurySet

In [26]:
%cd model_weights
!wget http://vllab1.ucmerced.edu/~wenbobao/DAIN/best.pth

/content/DAIN/model_weights
--2019-04-19 00:21:25--  http://vllab1.ucmerced.edu/~wenbobao/DAIN/best.pth
Resolving vllab1.ucmerced.edu (vllab1.ucmerced.edu)... 169.236.184.68
Connecting to vllab1.ucmerced.edu (vllab1.ucmerced.edu)|169.236.184.68|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 96319643 (92M)
Saving to: ‘best.pth’

best.pth            100%[===================>]  91.86M  28.9MB/s    in 3.2s    

2019-04-19 00:21:29 (28.9 MB/s) - ‘best.pth’ saved [96319643/96319643]



In [27]:
%cd ../MiddleBurySet
!wget http://vision.middlebury.edu/flow/data/comp/zip/other-color-allframes.zip
!unzip other-color-allframes.zip
!wget http://vision.middlebury.edu/flow/data/comp/zip/other-gt-interp.zip
!unzip other-gt-interp.zip
%cd ..

/content/DAIN/MiddleBurySet
--2019-04-19 00:21:33--  http://vision.middlebury.edu/flow/data/comp/zip/other-color-allframes.zip
Resolving vision.middlebury.edu (vision.middlebury.edu)... 140.233.20.14
Connecting to vision.middlebury.edu (vision.middlebury.edu)|140.233.20.14|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33671986 (32M) [application/zip]
Saving to: ‘other-color-allframes.zip’

other-color-allfram 100%[===================>]  32.11M  58.9MB/s    in 0.5s    

2019-04-19 00:21:33 (58.9 MB/s) - ‘other-color-allframes.zip’ saved [33671986/33671986]

Archive:  other-color-allframes.zip
  inflating: other-data/Beanbags/frame07.png  
  inflating: other-data/Beanbags/frame08.png  
  inflating: other-data/Beanbags/frame09.png  
  inflating: other-data/Beanbags/frame10.png  
  inflating: other-data/Beanbags/frame11.png  
  inflating: other-data/Beanbags/frame12.png  
  inflating: other-data/Beanbags/frame13.png  
  inflating: other-data/Beanbags/frame14.png 

In [19]:
!nvidia-smi

Fri Apr 19 00:08:51 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    16W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [28]:
#test that it works
!CUDA_VISIBLE_DEVICES=0 python demo_MiddleBury.py

revise the unique id to a random numer 88228
Namespace(SAVED_MODEL=None, arg='../model_weights/88228-Fri-Apr-19-00:21/args.txt', channels=3, datasetName='Vimeo_90K_interp', datasetPath='', dataset_split=97, debug=False, dtype=<class 'torch.cuda.FloatTensor'>, filter_size=4, force=False, log='../model_weights/88228-Fri-Apr-19-00:21/log.txt', netName='DAIN', no_date=False, save_path='../model_weights/88228-Fri-Apr-19-00:21', save_which=1, seed=1, time_step=0.5, uid=None, use_cuda=True, use_cudnn=1, workers=8)
cudnn is used
No weights loaded for Hourglass Network
The testing model weight is: ./model_weights/best.pth
The unique id for current testing is: 75682
Urban3
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.UpsamplingNearest2d is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/upsampling

In [30]:
# slow motion fun
# get rid of the uid flag if you don't want this to overwrite
!CUDA_VISIBLE_DEVICES=0 python demo_MiddleBury_slowmotion.py --netName DAIN_slowmotion --time_step 0.1 --uid 111

revise the unique id to a random numer 99501
Namespace(SAVED_MODEL=None, arg='../model_weights/99501-Fri-Apr-19-00:27/args.txt', channels=3, datasetName='Vimeo_90K_interp', datasetPath='', dataset_split=97, debug=False, dtype=<class 'torch.cuda.FloatTensor'>, filter_size=4, force=False, log='../model_weights/99501-Fri-Apr-19-00:27/log.txt', netName='DAIN_slowmotion', no_date=False, save_path='../model_weights/99501-Fri-Apr-19-00:27', save_which=1, seed=1, time_step=0.1, uid=None, use_cuda=True, use_cudnn=1, workers=8)
cudnn is used
No weights loaded for Hourglass Network
The testing model weight is: ./model_weights/best.pth
The unique id for current testing is: 28542
Urban3
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.UpsamplingNearest2d is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))
/usr/local/lib/python3.6/dist-packages/torch/nn/modules

In [32]:

%cd MiddleBurySet/other-result-author/111/Beanbags
!apt-get install imagemagick
!convert -delay 1 -loop 0 *.png beanbag.gif

[Errno 2] No such file or directory: 'MiddleBurySet/other-result-author/28542/Beanbags'
/content/DAIN/MiddleBurySet/other-result-author/28542/Beanbags
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono ghostscript gsfonts
  imagemagick-6-common imagemagick-6.q16 libcupsfilters1 libcupsimage2
  libdjvulibre-text libdjvulibre21 libgs9 libgs9-common libijs-0.35
  libjbig2dec0 liblqr-1-0 libmagickcore-6.q16-3 libmagickcore-6.q16-3-extra
  libmagickwand-6.q16-3 libnetpbm10 libwmf0.2-7 netpbm poppler-data
Suggested packages:
  fonts-noto ghostscript-x imagemagick-doc autotrace cups-bsd | lpr | lprng
  enscript gimp gnuplot grads hp2xx html2ps libwmf-bin mplayer povray radiance
  sane-utils texlive-base-bin transfig ufraw-bat

This builds 10x slomo gif at  `DAIN/MiddleBurySet/other-result-author/111/Beanbags/beanbag.gif`